<a href="https://colab.research.google.com/github/benjaminnigjeh/DIAMS/blob/main/hyperParameterOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Keras_Tuner
keras_tuner is a library that will be used for optimization of hyperparameters. The library should be installed running pip from the shell.

In [2]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.7 MB/s eta 0:00:00


# Import libraries and dependencies
google colab has already pre-installed versions of numpy, pandas, tensorflow, and sklearn.

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from keras_tuner.tuners import RandomSearch
from google.colab import files

# Import training and test datasets
the datasets should be downloaded to a local folder from GitHub repository, and uploaded to colab running environment

In [8]:
trainingDataset = files.upload_file('trainingDataset')

Saved training_dataset.csv to /content/trainingDataset


Saved test_dataset.csv to /content/testDataset


# Training Dataset wrangling

the sample labels are removed and numerical data transformed to numpy array

In [10]:
file_path = '/content/trainingDataset'
df = pd.read_csv(file_path)
Y = df.pop("target")
X = np.array(df)
y = np.array(Y)

# Helper functions to execute cycles of optimization

A model class is built wuth two dense layers which node numbers will be optimized

In [23]:
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(hp.Int("input_units_1", min_value =100, max_value =200, step =5), activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(hp.Int("input_units_2", min_value =100, max_value =300, step =5), activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')
    return model

tuner = RandomSearch(
    build_model,
    objective= "val_accuracy",
    max_trials= 100,
    executions_per_trial=1,
    directory= "logDir1"
    )
kf = KFold(n_splits=4, shuffle=True, random_state=100)


Reloading Tuner from logDir1/untitled_project/tuner0.json


# Optimization execution

a "for" cycle is executed to run the random search function of keras_tuner

In [24]:
for train_index, evaluation_index in kf.split(X):
    X_train, X_evaluation = X[train_index], X[evaluation_index]
    y_train, y_evaluation = y[train_index], y[evaluation_index]
    tuner.search(x=X_train,
                 y=y_train,
                 validation_data=(X_evaluation, y_evaluation))

Trial 14 Complete [00h 00m 02s]
val_accuracy: 0.4285714328289032

Best val_accuracy So Far: 0.8571428656578064
Total elapsed time: 00h 04m 46s

Search: Running Trial #15

Value             |Best Value So Far |Hyperparameter
100               |165               |input_units_1
265               |210               |input_units_2



KeyboardInterrupt: ignored